# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from sklearn.decomposition import TruncatedSVD
import joblib
import warnings
warnings.filterwarnings("ignore")
credits=pd.read_csv('/kaggle/input/movie-dataset/credits.csv')
keywords=pd.read_csv('/kaggle/input/movie-dataset/keywords.csv')
links=pd.read_csv('/kaggle/input/movie-dataset/links.csv')
links_small=pd.read_csv('/kaggle/input/movie-dataset/links_small.csv')
movies_metadata=pd.read_csv('/kaggle/input/movie-dataset/movies_metadata.csv')
ratings_small=pd.read_csv('/kaggle/input/movie-dataset/ratings_small.csv')

In [2]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [3]:
keywords.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [4]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links_small.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
movies_metadata.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [7]:
movie_ids=movies_metadata['id'].values
movie_ids_list=list(movie_ids)

In [8]:
import ast
def genres(text):
    genre=''
    text=ast.literal_eval(text)
    for i in text:
        genre+=i['name']+' '
    return genre[:-1]

In [9]:
def production_companies_names(text):
    company=''
    text=ast.literal_eval(text)
    for i in text:
        company+=i['name'].replace(" ",'')+" "
    return company[:-1]

In [10]:
def production_countries_names(text):
    country=''
    text=ast.literal_eval(text)
    for i in text:
        country+=i['name'].replace(" ",'')+" "
    return country[:-1]

In [11]:
def preprocess(movies,ratings):
    movies.drop(columns=['belongs_to_collection','homepage','tagline'],inplace=True)
    movies['inference_title']=movies['title']
    total_user_ratings=ratings.groupby(by=['userId']).count()['rating'].reset_index()
    active_user_ratings=ratings[ratings['userId'].isin(total_user_ratings[total_user_ratings['rating']>50]['userId'])]
    movies['genres_name']=movies['genres'].apply(genres)
    movies=movies[movies['genres_name']!='']
    movies=movies[movies['vote_count']>movies['vote_count'].quantile(0.8)]
    movies.drop(columns=['original_language','original_title','release_date','status','genres','video'],inplace=True)
    movies.dropna(inplace=True)
    movies['production_companies_names']=movies['production_companies'].apply(production_companies_names)
    movies['production_countries_names']=movies['production_countries'].apply(production_countries_names)
    movies.drop(columns=['production_companies','production_countries'],inplace=True)
    movies['adult']=movies['adult'].str.lower()
    movies['overview']=movies['overview'].str.lower()
    movies['title']=movies['title'].str.lower()
    movies['genres_name']=movies['genres_name'].str.lower()
    movies['production_companies_names']=movies['production_companies_names'].str.lower()
    movies['production_countries_names']=movies['production_countries_names'].str.lower()
    movies['adult']=movies['adult'].apply(lambda x: 'notadult' if x=='false' else 'adult')
    movies['training_text'] = movies['genres_name'] + ' ' + movies['title'] + ' ' + movies['overview'] + ' ' + movies['adult'] +' ' + movies['production_companies_names'] + ' ' + movies['production_countries_names']
    movies.drop(columns=['adult','overview','spoken_languages','production_companies_names','production_countries_names'],inplace=True)
    movies['title']=movies['title'].str.replace(" ",'')
    return movies,ratings



In [12]:
movies,ratings=preprocess(movies_metadata,ratings_small)

In [13]:
def content_model_train(movies,path):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    movie_embeddings = []

    for movie_description in movies['training_text']:
        tokens = tokenizer.encode(movie_description, truncation=True, padding=True, return_tensors='pt')
        with torch.no_grad():
            outputs = bert_model(tokens)
            embeddings = outputs['last_hidden_state'][:, 0, :]
            movie_embeddings.append(embeddings)

    movie_embeddings = torch.cat(movie_embeddings, dim=0)

    similarity_matrix = cosine_similarity(movie_embeddings, movie_embeddings)
    model_directory=path
    tokenizer.save_pretrained(model_directory)
    bert_model.save_pretrained(model_directory)
    np.save(f'{path}similarity_matrix.npy', similarity_matrix)
    torch.save(movie_embeddings, 'movie_embeddings.pt')
    return similarity_matrix,movie_embeddings

In [14]:
# similarity_matrix,movie_embeddings=content_model_train(movies,'/kaggle/working/')

In [55]:
def recommed_existing_movie(movie_name,movies,similarity_matrix=np.load('/kaggle/working/similarity_matrix.npy')):
    movie_name=movie_name.lower().replace(" ",'')
    input_movie_index = (movies[movies['title']==movie_name]['id']).index[0]
    similar_movies_indices = similarity_matrix[input_movie_index].argsort()[::-1]
    similar_movies_titles = movies.iloc[similar_movies_indices]['inference_title']
    similar_movies=movies.iloc[similar_movies_indices]
    print(f"Top Movies Similar to '{movies_metadata.iloc[input_movie_index]['title']}':")
#     print(similar_movies_titles)
    return similar_movies

In [32]:
similar_movies=recommed_existing_movie('heat',movies)

Top 5 Movies Similar to 'Father of the Bride Part II':
4                              Heat
946                   Lethal Weapon
948                 Lethal Weapon 3
120                      Disclosure
2307                   The Gauntlet
                   ...             
5856            Sorry if I Love You
3674                       Memories
4552    My Brother Is an Only Child
3881                       Yamakasi
2536                         Django
Name: inference_title, Length: 8527, dtype: object


In [69]:
def preprocess_new_movie(new_movie):
    new_movie['genres_name'] = new_movie['genres'].apply(genres)
    new_movie['production_companies_names'] = new_movie['production_companies'].apply(production_companies_names)
    new_movie['production_countries_names'] = new_movie['production_countries'].apply(production_countries_names)
    
    new_movie['adult'] = new_movie['adult'].str.lower()
    new_movie['overview'] = new_movie['overview'].str.lower()
    new_movie['title'] = new_movie['title'].str.lower()
    new_movie['genres_name'] = new_movie['genres_name'].str.lower()
    new_movie['production_companies_names'] = new_movie['production_companies_names'].str.lower()
    new_movie['production_countries_names'] = new_movie['production_countries_names'].str.lower()
    
    new_movie['adult'] = new_movie['adult'].apply(lambda x: 'notadult' if x == 'false' else 'adult')
    new_movie['training_text'] = new_movie['genres_name'] + ' ' + new_movie['title'] + ' ' + new_movie['overview'] + ' ' + new_movie['adult'] + ' ' + new_movie['production_companies_names'] + ' ' + new_movie['production_countries_names']
    
    new_movie['title'] = new_movie['title'].str.replace(" ", '')
    
    return new_movie
def generate_embedding(new_movie_description, tokenizer, bert_model):
    tokens = tokenizer.encode(new_movie_description, truncation=True, padding=True, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(tokens)
        embedding = outputs['last_hidden_state'][:, 0, :]
    return embedding
def recommend_new_movie(new_movie, movies, similarity_matrix_path='/kaggle/working/similarity_matrix.npy'):
    # Preprocess new movie data
    new_movie = preprocess_new_movie(new_movie)
    
    tokenizer = BertTokenizer.from_pretrained('/kaggle/working/')
    bert_model = BertModel.from_pretrained('/kaggle/working/')
    
    new_movie_embedding = generate_embedding(new_movie['training_text'].iloc[0], tokenizer, bert_model)
    
    similarity_matrix = np.load(similarity_matrix_path)
    
    existing_movie_embeddings = torch.load('movie_embeddings.pt')
    combined_embeddings = torch.cat([existing_movie_embeddings, new_movie_embedding], dim=0)
    new_similarity_matrix = cosine_similarity(combined_embeddings, combined_embeddings)
    new_movie_index = len(movies)
    similar_movies_indices = new_similarity_matrix[new_movie_index].argsort()[::-1][1:]
    similar_movies_title = movies.iloc[similar_movies_indices]['inference_title']
    similar_movies=movies.iloc[similar_movies_indices]
    return similar_movies

In [21]:
def collaborative_model_train(active_user_ratings):
    reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale=(0.5, 5), skip_lines=1)
    data = Dataset.load_from_df(active_user_ratings[['userId', 'movieId', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
    model = SVD(n_factors=20, random_state=42)
    model.fit(trainset)
    predictions = model.test(testset)
    svd_model = TruncatedSVD(n_components=100)  # Adjust the parameters based on your actual model
    joblib.dump(svd_model, 'svd_model.joblib')
    loaded_svd_model = joblib.load('svd_model.joblib')
#     print("Explained variance ratio:", sum(svd_model.explained_variance_ratio_))
    print(accuracy.rmse(predictions))
    return model

In [22]:
svd_model=collaborative_model_train(ratings)

RMSE: 0.8992
0.8992173663695584


In [23]:
def collaborative_inference(svd_model,user_id,ratings):
#     loaded_svd_model = joblib.load('/kaggle/working/svd_model.joblib')
    all_movie_ids = ratings['movieId'].unique()

    user_movie_combinations = [(user_id, movie_id, 0) for movie_id in all_movie_ids]

    predictions = svd_model.test(user_movie_combinations)

    recommendations = [(pred.iid, pred.est) for pred in predictions]

    recommendations.sort(key=lambda x: x[1], reverse=True)

    top_n_recommendations = recommendations[:10]
    print("Top Recommendations for User {}: {}".format(user_id, top_n_recommendations))

In [24]:
collaborative_inference(svd_model,20,ratings)

Top Recommendations for User 20: [(6016, 4.043046785805066), (1172, 3.9822702975659814), (1217, 3.9711546802364515), (2542, 3.961258356359624), (3462, 3.943831122251715), (1945, 3.8973316670302385), (1228, 3.89598051166504), (1192, 3.895721110173201), (2064, 3.8917534927166564), (745, 3.870570216231786)]


### hybrid

In [53]:
def hybrid(svd_model,movies,ratings,user_id,row,loaded_similarity_matrix=None):
    if(loaded_similarity_matrix==None):
        loaded_similarity_matrix = np.load('/kaggle/working/similarity_matrix.npy')
    movies.reset_index(inplace=True)
    movies.drop(columns='index',inplace=True)
    recommended_movies=recommed_existing_movie(row['title'].values[0],movies,loaded_similarity_matrix)
    all_movie_ids = ratings['movieId'].unique()
    user_movie_combinations = [(user_id, movie_id, 0) for movie_id in all_movie_ids]
    predictions = svd_model.test(user_movie_combinations)
    recommendations = [(pred.iid, pred.est) for pred in predictions]
    recommendations.sort(key=lambda x: x[1], reverse=True)
    recommendations_list=[ele[0] for ele in recommendations]
    recommended_movies=recommended_movies.set_index('id')
    recommendations_list = [str(idx).strip() for idx in recommendations_list]
    recommended_movies=recommended_movies[recommended_movies.index.isin(np.array(recommendations_list))]
    return recommended_movies


In [71]:
def recommend(movie_data,user_id,movies):
    if((movies['inference_title'].str.lower().replace(" ",'')==movie_data['title'].values[0]).any()):
        recommended_movies=hybrid(svd_model,movies,ratings,user_id,movie_data)['inference_title']
        return recommended_movies
    else:
        similar_movie_df=recommend_new_movie(new_movie_data, movies)
        return similar_movie_df

In [72]:
user_id=20
movie_name='heat'

new_movie_data = pd.DataFrame({
    'genres': ["[{'id': 18, 'name': 'Drama'}]"],
    'production_companies': ["[{'name': 'Warner Bros.'}]"],
    'production_countries': ["[{'name': 'United States of America'}]"],
    'adult': ['True'],
    'overview': ['Thirty years after the events of Blade Runner (1982), a new Blade Runner, L.A.P.D. Officer "K" (Ryan Gosling), unearths a long-buried secret that has the potential to plunge whats left of society into chaos. K\'s discovery leads him on a quest to find Rick Deckard (Harrison Ford), a former L.A.P.D. Blade Runner, who has been missing for thirty years.'],
    'title': ['Blade runner 2049'],
    'id': [99999]  # Just an example ID
})

movie=movies[movies['inference_title'].str.lower().replace(" ",'')==movie_name]
recommend(new_movie_data,user_id,movies)

,budget,id,imdb_id,popularity,poster_path,revenue,runtime,title,vote_average,vote_count,inference_title,genres_name,training_text
5178,47000000,20533,tt1075747,9.482806,/fhpl0NYcwQ8oUmur30L0ANOay4L.jpg,10539000.0,80.0,jonahhex,4.5,427.0,Jonah Hex,action western drama fantasy thriller,action western drama fantasy thriller jonah he...
3259,0,13696,tt0029870,9.937706,/fTo9qeUDLd75MNe7MxDlIgFR7pf.jpg,0.0,97.0,angelswithdirtyfaces,7.6,74.0,Angels with Dirty Faces,drama action crime,drama action crime angels with dirty faces in ...
864,40000000,18937,tt0120800,13.02535,/ePgK8gxGHVi1G3pYo0Geicbvur5.jpg,38172500.0,86.0,questforcamelot,6.9,193.0,Quest for Camelot,fantasy animation drama romance family,fantasy animation drama romance family quest f...
3236,0,15794,tt0042041,11.300976,/iXZdq7gep9BLlHa9cyGW8X9M9S2.jpg,0.0,114.0,whiteheat,7.8,92.0,White Heat,crime drama thriller,crime drama thriller white heat a psychopathic...
1161,0,11825,tt0095882,10.278957,/iwYu3FsGSNAaDgRC9i9Z7l1iVFm.jpg,19510371.0,90.0,policeacademy5:assignmentmiamibeach,4.7,195.0,Police Academy 5: Assignment Miami Beach,comedy crime,comedy crime police academy 5: assignment miam...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4552,5000000,9789,tt0846040,4.391819,/vhGgLFl5DFzbU842JAwmWjWUnSf.jpg,6463286.0,108.0,mybrotherisanonlychild,6.7,61.0,My Brother Is an Only Child,comedy,comedy my brother is an only child accio (elio...
2489,0,2750,tt0274309,12.655728,/2DJ3tP4yKYFodXXqVeLJVjE2Ssw.jpg,0.0,117.0,24hourpartypeople,6.8,109.0,24 Hour Party People,comedy drama,"comedy drama 24 hour party people in 1976, ton..."
3674,0,42994,tt0113799,4.648211,/rayHZ7QCXsjCcWLt7LKJKjGWG1E.jpg,0.0,113.0,memories,7.2,60.0,Memories,fantasy animation science fiction,fantasy animation science fiction memories in ...
7667,12560000,31359,tt0214289,4.052991,/yYRehBmPveMJbsmMvb1tg1SO7t8.jpg,42700000.0,105.0,wouldilietoyou?2,6.4,81.0,Would I Lie to You? 2,comedy,comedy would i lie to you? 2 dans le quartier ...
